In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor  
import warnings
import pickle
import os
import json

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [44]:
cols = ['ts', 'uid', 'id.orig_h', 'id.orig_p',
        'id.resp_h', 'id.resp_p', 'proto', 'service',
        'duration',  'orig_bytes', 'resp_bytes',
        'conn_state', 'local_orig', 'local_resp',
        'missed_bytes',  'history', 'orig_pkts',
        'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
        'tunnel_parents', 'label']

In [66]:
out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/ustc/Zeek/timeout60'
df = pd.read_csv(out_dir+"/USTC-TFC16_zeek_60.csv")
df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label
0,27991.284791,CPENHr72z4S7cEwb8,1.1.3.136,4248,1.2.191.218,443,tcp,-,-,-,-,OTH,F,F,0,D,1,118,0,0,-,normal
1,27991.284782,CF0yD74U8sFTNveBk7,1.1.150.6,11912,1.2.157.205,443,tcp,-,0.000020,0,413,OTH,F,F,0,^dA,1,52,1,465,-,normal
2,27991.284781,ClYGWPkrrlxehlLg3,1.1.105.11,51473,1.2.196.88,443,tcp,-,-,-,-,OTH,F,F,0,^d,0,0,1,888,-,normal
3,27991.284780,CM0T2K3vYv2HcJvFb8,1.1.16.104,45176,1.2.163.188,443,tcp,-,-,-,-,OTH,F,F,0,D,1,145,0,0,-,normal
4,27991.284779,CUH4T92e1GvKjpzqtj,1.1.176.72,48398,1.2.12.86,443,tcp,-,-,-,-,OTH,F,F,0,D,1,118,0,0,-,normal


In [2]:
def save_scores(timeout, meanScores, stdScore):
    results = {
        'Timeout': timeout,
        'Mean of all scores': meanScores,
        'Std of all Scores': stdScores

    }

    with open(f'../Checkpoints/MLP/MLP_ustc_zeek_{timeout}.json', 'w') as f:
        json.dump(results, f, indent=4)

In [3]:
timeouts = ['default', 0.5, 1, 2, 3, 4, 5, 6, 10, 30, 60]

# Training

In [4]:
best_f1 = 0
worst_f1 = 1
best_mean, worst_mean, best_std, worst_std = None, None, None, None

save=True


for timeout in timeouts:
    print("Processing timeout : ", timeout)
    if timeout =='default':
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/ustc/Zeek/{timeout}/USTC-TFC16_zeek_{timeout}.csv'
    else:
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/ustc/Zeek/timeout{timeout}/USTC-TFC16_zeek_{timeout}.csv'

    df = pd.read_csv(out_dir)
    
    df = df.drop(columns=['uid', 'id.orig_h', 'id.resp_h', 'tunnel_parents']) # tunnel_parents is empty

    # Handle missing values (if any)
    df.replace({'orig_bytes': '-'}, '0', inplace=True)
    df['orig_bytes'] = pd.to_numeric(df['orig_bytes'], errors='coerce')
    df['orig_bytes'] = df['orig_bytes'].fillna(0).astype('int64')

    df.replace({'resp_bytes': '-'}, '0', inplace=True)
    df['resp_bytes'] = pd.to_numeric(df['resp_bytes'], errors='coerce')
    df['resp_bytes'] = df['resp_bytes'].fillna(0).astype('int64')


    df.replace({'duration': '-'}, '0', inplace=True)
    df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
    df['duration'] = df['duration'].fillna(0).astype('float64')

    df['service'] = df['service'].replace('-', np.nan)
    df['history'] = df['history'].replace('-', np.nan)

    # Convert categorical variables to numerical using Label Encoding
    # Encode protocol and service types
    label_encoders = {}
    for column in ['proto', 'service', 'conn_state', 'history', 'local_orig', 'local_resp']:
        if column in df.columns:
            le = LabelEncoder()
            df[column] = le.fit_transform(df[column])
            label_encoders[column] = le

    # Split df into features and labels
    X = df.drop(columns=['label'])  # Assuming 'label' is the target variable
    y = df['label']
    
    accuracy, f1, precision, recall =[], [], [], []
    skf= StratifiedKFold(n_splits=5,random_state=None)
    skf.get_n_splits(X,y)
    
    for (train_index, test_index), i in zip(skf.split(X, y), range(5)):
        
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]


        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Initialize and train Extra Trees Classifier
        clf = MLPClassifier(random_state=123, solver='adam', max_iter=8000)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        
        f1Score = f1_score(y_true=y_test, y_pred=pred, average='macro')
        accScore=accuracy_score(y_test, pred)
        precScore = precision_score(y_test, pred, average='macro')
        recScrore = recall_score(y_test, pred, average='macro')
                             
        f1.append(f1Score)
        accuracy.append(accScore)
        precision.append(precScore)
        recall.append(recScrore)
        print('Fold: ', i, 'done!')

    meanScores, stdScores = {}, {}
    
    meanScores['f1Mean'] = np.array(f1).mean()
    meanScores['accMean'] = np.array(accuracy).mean()
    meanScores['recMean'] = np.array(recall).mean()
    meanScores['precMean'] = np.array(precision).mean()
    
    stdScores['f1Std'] = np.array(f1).std()
    stdScores['accStd'] = np.array(accuracy).std()
    stdScores['recStd'] = np.array(recall).std()
    stdScores['precStd'] = np.array(precision).std()
    
    print("Mean of all scores: ", meanScores)
    print("Std of all scores: ", stdScores)


    if save:
        save_scores(timeout, meanScores, stdScores)

    if meanScores['f1Mean'] > best_f1: 
        best_timeout = timeout
        best_mean = meanScores
        best_std = stdScores
        best_f1 = meanScores['f1Mean']
    
    if meanScores['f1Mean'] <= worst_f1: 
        
        worst_timeout = timeout
        worst_mean = meanScores
        worst_std = stdScores
        worst_f1 = meanScores['f1Mean']
               
    print('_______________________________________________')

Processing timeout :  default
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.8028120279604372, 'accMean': 0.936926840817615, 'recMean': 0.8006076205444259, 'precMean': 0.8259149617757391}
Std of all scores:  {'f1Std': 0.04851860333817631, 'accStd': 0.019372544338109275, 'recStd': 0.04750320810726693, 'precStd': 0.03579410264072459}
_______________________________________________
Processing timeout :  0.5
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.7827913330105177, 'accMean': 0.9294955836915474, 'recMean': 0.7892027376661199, 'precMean': 0.8121904297474065}
Std of all scores:  {'f1Std': 0.07218741719436458, 'accStd': 0.02219242323743128, 'recStd': 0.060293371287777016, 'precStd': 0.06387846354840995}
_______________________________________________
Processing timeout :  1
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scor

In [5]:
print("Best Timeout Combination: ", best_timeout)
print("Mean Scores (Best): ", best_mean)
print('Std Scores (Best):', best_std)

Best Timeout Combination:  default
Mean Scores (Best):  {'f1Mean': 0.8028120279604372, 'accMean': 0.936926840817615, 'recMean': 0.8006076205444259, 'precMean': 0.8259149617757391}
Std Scores (Best): {'f1Std': 0.04851860333817631, 'accStd': 0.019372544338109275, 'recStd': 0.04750320810726693, 'precStd': 0.03579410264072459}


In [6]:
print("worst Timeout Combination: ", worst_timeout)
print("Mean Scores (Worst): ", worst_mean)
print('Std Scores (Worst):', worst_std)

worst Timeout Combination:  60
Mean Scores (Worst):  {'f1Mean': 0.7741585475353988, 'accMean': 0.9292758550699002, 'recMean': 0.7964777652169939, 'precMean': 0.7849664715900773}
Std Scores (Worst): {'f1Std': 0.06315152698667863, 'accStd': 0.021340510160580257, 'recStd': 0.05212614888561606, 'precStd': 0.055254339162881465}


In [7]:

results = {
    'Best score': {
        'Best Timeout': best_timeout,
        'Mean Scores (Best)': best_mean,
        'Std Scores (Best)': best_std,
    },
    
    'Worst score': {
        'Worst Timeout': worst_timeout,
        'Mean Scores (Worst)': worst_mean,
        'Std Scores (Worst)': worst_std,
    },
    
    'Difference': {
        'Accuracy': (best_mean['accMean'] - worst_mean['accMean'])*100,
        'F1 Score': (best_mean['f1Mean'] - worst_mean['f1Mean'])*100,
        'Precision': (best_mean['precMean'] - worst_mean['precMean'])*100,
        'Recall': (best_mean['recMean'] - worst_mean['recMean'])*100
    }
}



with open('../results/MLP_ustc_zeek.json', 'w') as f:
    json.dump(results, f, indent=4)